In [17]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_v1.csv', sep=',')

Давайте определим число уникальных категорий в каждом столбце нашей таблицы melb_df. Для этого создадим вспомогательную таблицу unique_counts:

In [18]:
#создаём пустой список
unique_list = []
#пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    #создаем имя столбца (имя столбца, число уникальныъ значений)
    item = (col, melb_df[col].nunique(), melb_df[col].dtypes) #добавляем кортеж в список
    unique_list.append(item)
#создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Coulumn_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique', ignore_index = True)
#выводим её на экран
display(unique_counts)

,Coulumn_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,WeekdaySale,5,int64
3,Method,5,object
4,Regionname,8,object
5,Rooms,9,int64
6,Bathroom,9,int64
7,StreetType,11,object
8,Car,11,int64
9,MonthSale,12,int64


Самый простой способ преобразования столбцов к типу данных category — это использование уже знакомого нам метода astype(), в параметры которого достаточно передать строку 'category'.


In [19]:
#Для начала, выведем информацию о памяти, занимаемой текущей таблицей, с помощью метода info():
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Rooms          13580 non-null  int64  
 2   Type           13580 non-null  object 
 3   Method         13580 non-null  object 
 4   SellerG        13580 non-null  object 
 5   Date           13580 non-null  object 
 6   Distance       13580 non-null  float64
 7   Postcode       13580 non-null  int64  
 8   Bedroom        13580 non-null  int64  
 9   Bathroom       13580 non-null  int64  
 10  Car            13580 non-null  int64  
 11  BuildingArea   13580 non-null  float64
 12  CouncilArea    12211 non-null  object 
 13  Regionname     13580 non-null  object 
 14  Propertycount  13580 non-null  int64  
 15  StreetType     13580 non-null  object 
 16  WeekdaySale    13580 non-null  int64  
 17  Weekend        13580 non-null  int64  
 18  MonthS

None

In [20]:
#Сделаем преобразование столбцов к типу данных category:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique()<max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Suburb         13580 non-null  object  
 1   Rooms          13580 non-null  int64   
 2   Type           13580 non-null  category
 3   Method         13580 non-null  category
 4   SellerG        13580 non-null  category
 5   Date           13580 non-null  object  
 6   Distance       13580 non-null  float64 
 7   Postcode       13580 non-null  int64   
 8   Bedroom        13580 non-null  int64   
 9   Bathroom       13580 non-null  category
 10  Car            13580 non-null  int64   
 11  BuildingArea   13580 non-null  float64 
 12  CouncilArea    12211 non-null  category
 13  Regionname     13580 non-null  category
 14  Propertycount  13580 non-null  int64   
 15  StreetType     13580 non-null  category
 16  WeekdaySale    13580 non-null  category
 17  Weekend        13580 non-null  

None

У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их. Например, с помощью атрибута этого аксессора categories мы можем получить список уникальных категорий в столбце Regionname

In [21]:
print(melb_df['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


In [22]:
#А теперь посмотрим, каким образом столбец кодируется в виде чисел в памяти компьютера. Для этого можно воспользоваться атрибутом codes:

display(melb_df['Regionname'].cat.codes)

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

In [23]:
#С помощью метода аксессора rename_categories() можно легко переименовать текущие значения категорий. 
# Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.
#Рассмотрим на примере: переименуем категории признака типа постройки 
#Type — заменим их на полные названия (напомним, u — unit, h — house, t — townhouse).

melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])


0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

А теперь представим ситуацию, что появилась новая партия домов и теперь мы продаём и квартиры (flat). Создадим объект Series new_houses_types, в котором будем хранить типы зданий новой партии домов. Преобразуем тип new_houses_types в такой же тип, как и у столбца Type в таблице melb_data, и выведем результат на экран:

In [26]:
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

#тип данных category хранит только категории, которые были объявлены при его инициализации. 
# При встрече с новой, неизвестной ранее категорией, 
# этот тип превратит её в пустое значение, так как он просто не знает о существовании этой категории.

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

Решить эту проблему на самом деле не сложно. Можно добавить категорию flat в столбец Type с помощью метода акссесора cat add_categories(), в который достаточно просто передать имя новой категории:

In [27]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)


0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

In [127]:
#Задание 5.2
#При преобразовании столбцов таблицы о недвижимости к типу category мы оставили без внимания 
#столбец Suburb (пригород). Давайте исправим это.
#С помощью метода info() узнайте, сколько памяти занимает таблица melb_df.
#Преобразуйте признак Suburb следующим образом: оставьте в столбце только 119 наиболее популярных пригородов, 
#остальные замените на 'other'.
#Приведите данные в столбце Suburb к категориальному типу.
import pandas as pd
melb_data = pd.read_csv('data/melb_data.csv', sep=',')
melb_df=melb_data.copy()
popular_suburb=melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb']=melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
display(melb_df.info())
display(melb_data.info())
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  category
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  object  
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  object  
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  float64 
 12  Bathroom       13580 non-null  float64 
 13  Car            13580 non-null  float64 
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  float64 
 17  CouncilArea    12211 non-null  

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          13580 non-null  int64  
 1   Suburb         13580 non-null  object 
 2   Address        13580 non-null  object 
 3   Rooms          13580 non-null  int64  
 4   Type           13580 non-null  object 
 5   Price          13580 non-null  float64
 6   Method         13580 non-null  object 
 7   SellerG        13580 non-null  object 
 8   Date           13580 non-null  object 
 9   Distance       13580 non-null  float64
 10  Postcode       13580 non-null  int64  
 11  Bedroom        13580 non-null  float64
 12  Bathroom       13580 non-null  float64
 13  Car            13580 non-null  float64
 14  Landsize       13580 non-null  float64
 15  BuildingArea   13580 non-null  float64
 16  YearBuilt      13580 non-null  float64
 17  CouncilArea    12211 non-null  object 
 18  Lattit

None